In [ ]:
{
  "nbformat": 4,
  "nbformat_minor": 0,
  "metadata": {
    "colab": {
      "provenance": []
    },
    "kernelspec": {
      "name": "python3",
      "display_name": "Python 3"
    },
    "language_info": {
      "name": "python"
    }
  },
  "cells": [
    {
      "cell_type": "markdown",
      "source": [
        "# Disease Analyzer reveals spread trends, risk factors, and hotspot detection"
      ],
      "metadata": {
        "id": "1Yr92PyRxUpp"
      }
    },
    {
      "cell_type": "markdown",
      "source": [
        "Imports\n"
      ],
      "metadata": {
        "id": "BMd7-LSXxkCC"
      }
    },
    {
      "cell_type": "code",
      "execution_count": 1,
      "metadata": {
        "id": "VtFpRuP4wt2a"
      },
      "outputs": [],
      "source": [
        "import gradio as gr\n",
        "import numpy as np\n",
        "import pandas as pd\n",
        "import matplotlib.pyplot as plt\n",
        "import seaborn as sns"
      ]
    },
    {
      "cell_type": "markdown",
      "source": [
        " Data Generator\n"
      ],
      "metadata": {
        "id": "Fg3HctXex0rE"
      }
    },
    {
      "cell_type": "code",
      "source": [
        "def generate_disease_data(n=5000):\n",
        "    np.random.seed(42)\n",
        "\n",
        "    df = pd.DataFrame({\n",
        "        \"population_density\": np.random.randint(200, 15000, n),\n",
        "        \"infection_rate\": np.random.uniform(0.2, 25, n),\n",
        "        \"recovery_rate\": np.random.uniform(40, 98, n),\n",
        "        \"death_rate\": np.random.uniform(0.1, 5, n),\n",
        "        \"vaccination_rate\": np.random.uniform(20, 95, n),\n",
        "        \"mobility_index\": np.random.uniform(10, 100, n),\n",
        "        \"air_quality_index\": np.random.randint(50, 300, n),\n",
        "        \"avg_temperature\": np.random.uniform(5, 45, n),\n",
        "        \"humidity\": np.random.uniform(20, 90, n),\n",
        "        \"hospital_beds_per_1000\": np.random.uniform(0.5, 8, n),\n",
        "        \"icu_capacity\": np.random.uniform(5, 80, n),\n",
        "        \"doctors_per_1000\": np.random.uniform(0.5, 5, n),\n",
        "        \"testing_rate\": np.random.uniform(100, 5000, n),\n",
        "        \"region\": np.random.choice(\n",
        "            [\"North\", \"South\", \"East\", \"West\", \"Central\"], n\n",
        "        )\n",
        "    })\n",
        "\n",
        "    # Risk Score Formula\n",
        "    df[\"risk_score\"] = (\n",
        "        (df[\"infection_rate\"] * 0.4)\n",
        "        + (df[\"population_density\"] * 0.0003)\n",
        "        - (df[\"vaccination_rate\"] * 0.2)\n",
        "        + (df[\"mobility_index\"] * 0.1)\n",
        "        + (df[\"air_quality_index\"] * 0.05)\n",
        "    )\n",
        "\n",
        "    # Risk Classification\n",
        "    def classify_risk(score):\n",
        "        if score < 15:\n",
        "            return \"Low\"\n",
        "        elif score < 30:\n",
        "            return \"Medium\"\n",
        "        else:\n",
        "            return \"High\"\n",
        "\n",
        "    df[\"risk_level\"] = df[\"risk_score\"].apply(classify_risk)\n",
        "\n",
        "    return df\n"
      ],
      "metadata": {
        "id": "v7n1PLytx78Y"
      },
      "execution_count": 2,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "source": [
        "Analysis Function"
      ],
      "metadata": {
        "id": "8ZGdcDHsyDHu"
      }
    },
    {
      "cell_type": "code",
      "source": [
        "def analyze_disease(risk_filter):\n",
        "    df = generate_disease_data()\n",
        "\n",
        "    if risk_filter != \"All\":\n",
        "        df = df[df[\"risk_level\"] == risk_filter]\n",
        "\n",
        "    # 1️⃣ Risk Distribution\n",
        "    fig1, ax1 = plt.subplots()\n",
        "    sns.countplot(data=df, x=\"risk_level\", ax=ax1)\n",
        "    ax1.set_title(\"Disease Risk Distribution\")\n",
        "\n",
        "    # 2️⃣ Regional Hotspots\n",
        "    fig2, ax2 = plt.subplots()\n",
        "    sns.countplot(data=df, x=\"region\", hue=\"risk_level\", ax=ax2)\n",
        "    ax2.set_title(\"Regional Disease Hotspots\")\n",
        "\n",
        "    # 3️⃣ Correlation Heatmap\n",
        "    fig3, ax3 = plt.subplots(figsize=(10, 6))\n",
        "    corr = df[[\n",
        "        \"population_density\", \"infection_rate\", \"recovery_rate\",\n",
        "        \"death_rate\", \"vaccination_rate\", \"mobility_index\",\n",
        "        \"air_quality_index\", \"hospital_beds_per_1000\"\n",
        "    ]].corr()\n",
        "\n",
        "    sns.heatmap(corr, annot=True, cmap=\"coolwarm\", ax=ax3)\n",
        "    ax3.set_title(\"Correlation Matrix\")\n",
        "\n",
        "    return fig1, fig2, fig3, df.head(15)\n"
      ],
      "metadata": {
        "id": "ExyxKJNoyBwR"
      },
      "execution_count": 3,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "source": [
        "Gradio Interface"
      ],
      "metadata": {
        "id": "e1wYjI6gyNkR"
      }
    },
    {
      "cell_type": "code",
      "source": [
        "interface = gr.Interface(\n",
        "    fn=analyze_disease,\n",
        "    inputs=gr.Dropdown(\n",
        "        choices=[\"All\", \"Low\", \"Medium\", \"High\"],\n",
        "        value=\"All\",\n",
        "        label=\"Select Risk Level\"\n",
        "    ),\n",
        "    outputs=[\n",
        "        gr.Plot(label=\"Risk Level Distribution\"),\n",
        "        gr.Plot(label=\"Regional Hotspots\"),\n",
        "        gr.Plot(label=\"Correlation Analysis\"),\n",
        "        gr.Dataframe(label=\"Sample Dataset\")\n",
        "    ],\n",
        "    title=\"Advanced Disease Spread Analyzer\",\n",
        "    description=(\n",
        "        \"This system analyzes large-scale disease spread data, \"\n",
        "        \"identifies hotspots, evaluates healthcare capacity, \"\n",
        "        \"and visualizes key risk factors.\"\n",
        "    )\n",
        ")\n",
        "\n",
        "interface.launch()"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 646
        },
        "id": "kRP5XajRyTee",
        "outputId": "b012459d-9b13-4c11-afc5-8c7afe62ba72"
      },
      "execution_count": 4,
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).\n",
            "\n",
            "Colab notebook detected. To show errors in colab notebook, set debug=True in launch()\n",
            "* Running on public URL: https://2b3474d7d4fa977f3d.gradio.live\n",
            "\n",
            "This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)\n"
          ]
        },
        {
          "output_type": "display_data",
          "data": {
            "text/plain": [
              "<IPython.core.display.HTML object>"
            ],
            "text/html": [
              "<div><iframe src=\"https://2b3474d7d4fa977f3d.gradio.live\" width=\"100%\" height=\"500\" allow=\"autoplay; camera; microphone; clipboard-read; clipboard-write;\" frameborder=\"0\" allowfullscreen></iframe></div>"
            ]
          },
          "metadata": {}
        },
        {
          "output_type": "execute_result",
          "data": {
            "text/plain": []
          },
          "metadata": {},
          "execution_count": 4
        }
      ]
    }
  ]
}